# Exploring the Galaxy Zoo data

From here: https://data.galaxyzoo.org

In [1]:
import pandas as pd

### Read in the data and select galaxies that were classified as spiral or elliptical

In [2]:
#df = pd.read_csv('GalaxyZoo1_DR_table2.csv') #Galazy Zoo 1
df = pd.read_csv('zoo2MainSpecz.csv', dtype={'specobjid':'str','dr8objid':'str','dr7objid':'str'}) #Galaxy Zoo 2, larger galaxies
print(df.shape)

(243500, 233)


In [3]:
#dfKnown = df.loc[(df['UNCERTAIN'] == 0) & (df['P_EDGE'] < 0.1)]
dfSpiral = df.loc[df['t04_spiral_a08_spiral_flag'] == 1]
dfSmooth = df.loc[df['t01_smooth_or_features_a01_smooth_flag'] == 1]
print(dfSpiral.shape)
print(dfSmooth.shape)

(32059, 233)
(27065, 233)


In [4]:
dfSpiral

,specobjid,dr8objid,dr7objid,ra,dec,rastring,decstring,sample,gz2class,total_classifications,...,t11_arms_number_a36_more_than_4_fraction,t11_arms_number_a36_more_than_4_weighted_fraction,t11_arms_number_a36_more_than_4_debiased,t11_arms_number_a36_more_than_4_flag,t11_arms_number_a37_cant_tell_count,t11_arms_number_a37_cant_tell_weight,t11_arms_number_a37_cant_tell_fraction,t11_arms_number_a37_cant_tell_weighted_fraction,t11_arms_number_a37_cant_tell_debiased,t11_arms_number_a37_cant_tell_flag
0,1802674929645152256,NaN,588017703996096547,160.990400,11.703790,10:43:57.70,+11:42:13.6,original,SBb?t,44,...,0.225,0.225,0.225000,0,10,10.0,0.250,0.250,0.250000,0
2,1489568922213574656,NaN,587735695913320507,210.802200,54.348953,14:03:12.53,+54:20:56.2,original,Sc+t,46,...,0.651,0.651,0.651000,0,3,3.0,0.070,0.070,0.070000,0
6,1809324500555163648,1237661948639772687,588017702391578633,175.244810,11.471134,11:40:58.75,+11:28:16.1,original,Sb+t,45,...,0.744,0.744,0.744000,0,6,6.0,0.140,0.140,0.140000,0
8,1907389949162842112,NaN,588017704545812500,190.985750,13.126677,12:43:56.58,+13:07:36.0,original,Sc?t,43,...,0.262,0.262,0.262000,0,17,17.0,0.405,0.405,0.405000,0
13,3132375113912051712,NaN,587742615095935051,234.132000,16.607780,15:36:31.68,+16:36:28.0,original,Sb+t,46,...,0.585,0.584,0.584000,0,14,14.0,0.341,0.342,0.342000,0
14,876021909865654272,1237653617475452940,588009371227258884,180.550870,62.137238,12:02:12.21,+62:08:14.1,original,Sb+t,39,...,0.784,0.795,0.795000,0,6,5.5,0.162,0.151,0.151000,0
15,1178913180705908736,NaN,587733410447491082,218.195200,49.457912,14:32:46.85,+49:27:28.5,original,Sc?t,45,...,0.316,0.316,0.316000,0,20,20.0,0.526,0.526,0.526000,0
16,379506367435663360,NaN,587724648188543033,192.056880,-3.332845,12:48:13.65,-03:19:58.2,original,SBb2l(o),42,...,0.000,0.000,0.000000,0,3,3.0,0.107,0.111,0.111000,0
19,2221553680774096896,1237664668973531242,587738947748626521,159.067700,37.324688,10:36:16.25,+37:19:28.9,original,Sc+t,44,...,0.571,0.571,0.571000,0,9,9.0,0.214,0.214,0.214000,0
21,1146264831163131904,1237657590858776636,587731869633871916,179.117230,55.125233,11:56:28.14,+55:07:30.8,original,Sb?t,45,...,0.381,0.381,0.381000,0,17,17.0,0.405,0.405,0.405000,0


### Pull thumbnails from SDSS using [Image Cutout](http://skyserver.sdss.org/dr12/en/help/docs/api.aspx#imgcutout) 

*This is what they used for Galaxy Zoo; see [here](http://adsabs.harvard.edu/abs/2011MNRAS.410..166L)*

In [5]:
from astroquery.sdss import SDSS
from astropy import coordinates as coords
from astropy import units as u

import requests 
import shutil  

/Users/ageller/anaconda/envs/py36/lib/python3.6/site-packages/astroquery/sdss/__init__.py:29: UserWarning: Experimental: SDSS has not yet been refactored to have its API match the rest of astroquery (but it's nearly there).
  warnings.warn("Experimental: SDSS has not yet been refactored to have its API "


In [6]:
def getImage(pos, fname='images/test.jpg'):
    url='http://skyserver.sdss.org/dr12/SkyserverWS/ImgCutout/getjpeg?'+\
        'ra='+str(pos.ra.degree)+'&dec='+str(pos.dec.degree)+'&width=424&height=424'

    #url='http://skyserver.sdss.org/dr12/SkyserverWS/ImgCutout/getjpeg?ra=224.5941&dec=-1.09&width=512&height=512&opt=OG'
    print(url)
    r = requests.get(url, stream=True)
    print(r)
    if r.status_code == 200:
        with open(fname, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)  

In [7]:
def addToOutput(df,i, output):
    num = len(output['id'])
    output['id'].append(num)
    fname = 'images/GZ2_'+str(num).zfill(5)+'.jpg'
    output['image'].append(fname)
    for key in output:
        if (key != 'id' and key != 'image'):
            output[key].append(df[key].values[i])

    print(fname, output['t04_spiral_a08_spiral_debiased'][-1],  output['t01_smooth_or_features_a01_smooth_debiased'][-1])
    
    #pos = coords.SkyCoord(dfKnown['RA'].values[i]+' '+dfKnown['DEC'].values[i], unit=(u.hourangle, u.deg), frame='icrs')
    pos = coords.SkyCoord(output['rastring'][-1]+' '+output['decstring'][-1], unit=(u.hourangle, u.deg), frame='icrs')
    getImage(pos, fname)

    return output

In [8]:
output = {
    'id':[],
    'image':[],
    'specobjid':[], #SDSS
    'rastring':[],
    'decstring':[],
    #'t04_spiral_a08_spiral_flag':[],
    't04_spiral_a08_spiral_debiased':[],
    #'t01_smooth_or_features_a01_smooth_flag':[],
    't01_smooth_or_features_a01_smooth_debiased':[],
    }

Ngal = 500 #number of galaxy PAIRS, so total number in file will be 2*Ngal
for i in range(Ngal):
    output = addToOutput(dfSpiral,i, output)
    output = addToOutput(dfSmooth ,i, output)


images/GZ2_00000.jpg 0.9520000000000001 0.002
http://skyserver.sdss.org/dr12/SkyserverWS/ImgCutout/getjpeg?ra=160.99041666666665&dec=11.703777777777777&width=424&height=424
<Response [200]>
images/GZ2_00001.jpg 0.222 0.8821604
http://skyserver.sdss.org/dr12/SkyserverWS/ImgCutout/getjpeg?ra=187.66937499999997&dec=9.015666666666666&width=424&height=424
<Response [200]>
images/GZ2_00002.jpg 1.0 0.0
http://skyserver.sdss.org/dr12/SkyserverWS/ImgCutout/getjpeg?ra=210.8022083333333&dec=54.34894444444445&width=424&height=424
<Response [200]>
images/GZ2_00003.jpg 0.0 0.8690000000000001
http://skyserver.sdss.org/dr12/SkyserverWS/ImgCutout/getjpeg?ra=153.4605&dec=38.76488888888889&width=424&height=424
<Response [200]>
images/GZ2_00004.jpg 0.977 0.0
http://skyserver.sdss.org/dr12/SkyserverWS/ImgCutout/getjpeg?ra=175.24479166666666&dec=11.47113888888889&width=424&height=424
<Response [200]>
images/GZ2_00005.jpg 0.0 0.8
http://skyserver.sdss.org/dr12/SkyserverWS/ImgCutout/getjpeg?ra=125.14870833333

<Response [200]>
images/GZ2_00915.jpg 0.0 0.97386116
http://skyserver.sdss.org/dr12/SkyserverWS/ImgCutout/getjpeg?ra=153.68887499999997&dec=38.981&width=424&height=424
<Response [200]>
images/GZ2_00916.jpg 1.0 0.0
http://skyserver.sdss.org/dr12/SkyserverWS/ImgCutout/getjpeg?ra=175.976375&dec=10.784694444444444&width=424&height=424
<Response [200]>
images/GZ2_00917.jpg 0.0 0.8458823000000001
http://skyserver.sdss.org/dr12/SkyserverWS/ImgCutout/getjpeg?ra=126.107625&dec=4.247694444444445&width=424&height=424
<Response [200]>
images/GZ2_00918.jpg 1.0 0.0
http://skyserver.sdss.org/dr12/SkyserverWS/ImgCutout/getjpeg?ra=162.11370833333334&dec=26.583805555555553&width=424&height=424
<Response [200]>
images/GZ2_00919.jpg 0.0 0.86305034
http://skyserver.sdss.org/dr12/SkyserverWS/ImgCutout/getjpeg?ra=224.75249999999997&dec=43.407694444444445&width=424&height=424
<Response [200]>
images/GZ2_00920.jpg 0.917 0.08800000000000001
http://skyserver.sdss.org/dr12/SkyserverWS/ImgCutout/getjpeg?ra=226.012

### Write this to a json file to be read in on website

In [9]:
pd.DataFrame(output).to_json('GZ2data.json',orient='records')